In [0]:
#import libareries
import math
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn


from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from keras.optimizers import Adam, Adadelta
from keras.layers import LSTM, SimpleRNN, Dense, PReLU, Dropout, Activation
from sklearn import model_selection
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from keras import backend as K

if 'tensorflow' == K.backend():
  import tensorflow as tf
  from keras.backend.tensorflow_backend import set_session
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
  config.gpu_options.visible_device_list = "0"
  set_session(tf.Session(config=config))

Using TensorFlow backend.


In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 24.2MB/s 


In [0]:
import os

# choose a local (colab) directory to store the data.
local_root_path = os.path.expanduser("~/Business Intelligence")
try:
  os.makedirs(local_root_path)
except: pass

def ListFolder(google_drive_id, destination):
  file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % google_drive_id}).GetList()
  counter = 0
  for f in file_list:
    # If it is a directory then, create the dicrectory and upload the file inside it
    if f['mimeType']=='application/vnd.google-apps.folder': 
      folder_path = os.path.join(destination, f['title'])
      os.makedirs(folder_path)
      print('creating directory {}'.format(folder_path))
      ListFolder(f['id'], folder_path)
    else:
      fname = os.path.join(destination, f['title'])
      f_ = drive.CreateFile({'id': f['id']})
      f_.GetContentFile(fname)
      counter += 1
  print('{} files were uploaded in {}'.format(counter, destination))

ListFolder("Path to the Drive Folder", local_root_path)

In [0]:
!ls ~/"Business Intelligence"

2_sf_models.ipynb		       3_sf_models_plot.ipynb	 x_train
3a_sf_models-deeplearning-colab.ipynb  crimeSF_NN_logodds.ipynb  y_test
3a_sf_models-deeplearning_TPU.ipynb    test_data		 y_train
3_sf_models.ipynb		       x_test


In [0]:
#load dataframes from preproc
x_train = pd.read_pickle('~/Business Intelligence/x_train')
x_test = pd.read_pickle('~/Business Intelligence/x_test')
y_test = pd.read_pickle('~/Business Intelligence/y_test')
y_train = pd.read_pickle('~/Business Intelligence/y_train')
test_data = pd.read_pickle('~/Business Intelligence/test_data')

labels = np.array(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'])

In [0]:
def modelToCsv_deeplearning(model):
    """Die Methode schreibt predictions des übergebenen Modells in eine CSV Datei Namens 'submit.csv'.
    Die CSV Datei hat das von Kaggle gewünschte Format."""
    #Droppe Column Id, um die gleichen Features auf den Testdaten wie auf den Trainingsdaten zu haben
    test_daten = test_data.drop(['Id'], axis = 1)
    #füge eine Dimension hinzu, um prediction zu ermöglichen
    test_daten = test_daten.values
    test_daten = test_daten.reshape((test_daten.shape[0], 1, test_daten.shape[1]))
    #Predicte das Ergebnis
    predicted = model.predict_proba(test_daten)

    
    result = pd.DataFrame(predicted, columns = labels)
    result.to_csv('submit.csv',index=True, index_label='Id')

In [0]:
#Zusätzliche Datapreparation, um es im Neuronalen Netz nutzen zu können
#Nur einmal Ausführen
y_train = to_categorical(y_train)
x_train = x_train.values

y_test = to_categorical(y_test)
x_test = x_test.values

In [0]:
# Die Erstellung des tiefen neuronalen Netzes
def create_NeuralNet(layer, nodes):
  
  model = Sequential()
  model.add(Dense(nodes, init='glorot_uniform', use_bias=True))
  model.add(PReLU())
  model.add(Dropout(0.2))
  
  for i in range(layer):
    model.add(Dense(nodes, init='glorot_uniform', use_bias=True))
    model.add(PReLU())
    model.add(Dropout(0.2))


  model.add(Dense(39, activation='softmax'))


  model.compile(loss='categorical_crossentropy', optimizer=Adadelta())
  
  return model

model = create_NeuralNet(30, 39)

W0803 16:34:01.561525 140444664514432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(39, use_bias=True, kernel_initializer="glorot_uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(39, use_bias=True, kernel_initializer="glorot_uniform")`
  if __name__ == '__main__':
W0803 16:34:01.592282 140444664514432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:
#options
model.fit(x_train, 
          y_train,
          validation_data=(x_test, y_test),
          batch_size=40000,
          epochs=200)

In [0]:

x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

In [0]:
def create_LSTM(layer, nodes):
  
  model = Sequential()

  model.add(LSTM(nodes,go_backwards= True, use_bias=True, kernel_initializer='glorot_uniform', input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
  model.add(PReLU())
  model.add(Dropout(0.2))
  

  for i in range(layer):
    model.add(LSTM(nodes, use_bias=True, kernel_initializer='glorot_uniform', input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(PReLU())
    model.add(Dropout(0.2))



  model.add(LSTM(39, use_bias=True, kernel_initializer='glorot_uniform', input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=False))
  model.add(PReLU())
  model.add(Dropout(0.2))
  model.add(Dense(39, activation='softmax'))


  model.compile(loss='categorical_crossentropy', optimizer=Adadelta())
  
  return model

model = create_LSTM(50, 70)

In [0]:
#options
model.fit(x_train, 
          y_train,
          validation_data=(x_test, y_test),
          batch_size=40000,
          epochs=200)

Train on 700527 samples, validate on 175132 samples
Epoch 1/200
700527/700527 [==============================] - 96s 137us/step - loss: 3.6496 - val_loss: 3.6331
Epoch 2/200
700527/700527 [==============================] - 35s 50us/step - loss: 3.6191 - val_loss: 3.6026
Epoch 3/200
700527/700527 [==============================] - 34s 49us/step - loss: 3.5887 - val_loss: 3.5724
Epoch 4/200
700527/700527 [==============================] - 35s 49us/step - loss: 3.5588 - val_loss: 3.5428
Epoch 5/200
700527/700527 [==============================] - 35s 49us/step - loss: 3.5294 - val_loss: 3.5137
Epoch 6/200
700527/700527 [==============================] - 35s 50us/step - loss: 3.5006 - val_loss: 3.4852
Epoch 7/200
700527/700527 [==============================] - 35s 50us/step - loss: 3.4723 - val_loss: 3.4573
Epoch 8/200
700527/700527 [==============================] - 35s 49us/step - loss: 3.4447 - val_loss: 3.4299
Epoch 9/200
700527/700527 [==============================] - 35s 49us/step 

KeyboardInterrupt: ignored